Summarize a long document by chunking and summarizing parts.
Uses aynchronous calls to the API.
Adapted from LangChain [Map-Reduce summary](https://langchain.readthedocs.io/en/stable/_modules/langchain/chains/mapreduce.html).

In [1]:
import trio

In [2]:
from minichain import TemplatePrompt, show_log, start_chain

Prompt that asks LLM to produce a bash command.

In [3]:
class SummaryPrompt(TemplatePrompt[str]):
    template_file = "summary.pmpt.tpl"

In [4]:
def chunk(f):
    "Split a documents into 4800 character overlapping chunks"
    text = open(f).read().replace("\n\n", "\n")
    chunks = []
    W = 4000
    O = 800
    for i in range(4):
        if i * W > len(text):
            break
        chunks.append({"text": text[i * W : (i + 1) * W + O]})
    return chunks

In [5]:
with start_chain("summary") as backend:
    prompt = SummaryPrompt(backend.OpenAI())
    list_prompt = prompt.map()

    # Map - Summarize each chunk in parallel
    out = trio.run(list_prompt.arun, chunk("../state_of_the_union.txt"))

    # Reduce - Summarize the summarized chunks
    print(prompt({"text": "\n".join(out)}))

INFO     2023-02-25 20:42:16.679: async_openai.client:__init__: OpenAI Client initialized: https://api.openai.com/v1


INFO     2023-02-25 20:42:16.679: async_openai.client:__init__: OpenAI Client initialized: https://api.openai.com/v1


 In response to Russia's aggression in Ukraine, President Biden has taken action to provide military, economic, and humanitarian assistance to Ukraine, impose economic sanctions on Russia, and close off American airspace to Russian flights. He has also passed the American Rescue Plan to provide economic relief to Americans and the Bipartisan Innovation Act to level the playing field with China and other competitors. This bill will create 10,000 new jobs and increase Intel's investment to $100 billion. President Biden is also fighting inflation by capping the cost of insulin at $35 a month and allowing Medicare to negotiate lower prices for prescription drugs.


In [6]:
SummaryPrompt().show(
    {
        "text": "One way to fight inflation is to drive down wages and make Americans poorer."
    },
    "Make Americans poorer",
)

HTML(html='\n<!-- <link rel="stylesheet" href="https://cdn.rawgit.com/Chalarangelo/mini.css/v3.0.1/dist/mini-default.min.css"> -->\n <main class="container">\n\n<h3>SummaryPrompt</h3>\n\n<dl>\n  <dt>Input:</dt>\n  <dd>\n<div class="highlight"><pre><span></span><span class="p">{</span><span class="s1">&#39;text&#39;</span><span class="p">:</span> <span class="s1">&#39;One way to fight inflation is to drive down wages and make Americans poorer.&#39;</span><span class="p">}</span>\n</pre></div>\n\n\n  </dd>\n\n  <dt> Full Prompt: </dt>\n  <dd>\n    <details>\n      <summary>Prompt</summary>\n      <p>Write a concise summary of the following:<br><br><br>"<div style=\'color:red\'>One way to fight inflation is to drive down wages and make Americans poorer.</div>"<br><br><br>CONCISE SUMMARY:</p>\n    </details>\n  </dd>\n\n  <dt> Response: </dt>\n  <dd>\n    Make Americans poorer\n  </dd>\n\n  <dt>Value:</dt>\n  <dd>\n<div class="highlight"><pre><span></span><span class="n">Make</span> <span class="n">Americans</span> <span class="n">poorer</span>\n</pre></div>\n\n  </dd>\n</main>\n')

In [7]:
show_log("summary.log")

6174bdb9-8e7d-4859-9c91-d36235b2ad41
└── <class '__main__.SummaryPrompt'>/1 ⇒ started 2023-02-26 01:42:16Z ⧖ 3.979s
    ├── Input Function/2/1 ⇒ started 2023-02-26 01:42:16Z ⧖ 0.001s
    │   ├── input: 
    │   │   └── text: r economy. The Ruble has lost 30% of its value. ⏎
    │   │       The Russian stock market has lost 40% of its value and trading remains suspended. Russia’s economy is reeling and Putin alone is to blame. ⏎
    │   │       Together with our allies we are providing support to the Ukrainians in their fight for freedom. Military assistance. Economic assistance. Humanitarian assistance. ⏎
    │   │       We are giving more than $1 Billion in direct assistance to Ukraine. ⏎
    │   │       And we will continue to aid the Ukrainian people as they defend their country and to help ease their suffering.  ⏎
    │   │       Let me be clear, our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  ⏎
    │   │       Our forces are not going to